In [1]:
import numpy as np
import sys
import os
import hashlib
import json
import array
from typing import Any, Dict, List, Optional
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import oracledb
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")
from tqdm import TqdmExperimentalWarning
warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)

.env檔案內容
```
username = 'HR'
password = 'oracle'
dsn = 'localhost:1521/freepdb1'
COMPARTMENT_OCID =
OPENAI_API_KEY = ''
```

In [2]:
# 加載環境變量
load_dotenv()
username = os.getenv("username")
password = os.getenv("password")
dsn = os.getenv("dsn")
COMPARTMENT_OCID = os.getenv("Compartment_OCID")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# 建立Oracle資料庫連接
try:
    client = oracledb.connect(user=username, password=password, dsn=dsn)
    print("資料庫連接成功！")
except Exception as e:
    print("資料庫連接失敗！", e)
    sys.exit(1)

# 表存在刪除,不存在則創建
table_name = "VECTOR_PDF_DEMO"
with client.cursor() as cursor:
    cursor.execute(f"""
        DECLARE
            table_count NUMBER;
        BEGIN
            SELECT COUNT(*)
            INTO table_count
            FROM all_tables
            WHERE table_name = :table_name
              AND owner = :owner;
            IF table_count > 0 THEN
                EXECUTE IMMEDIATE 'DROP TABLE {table_name}';
            END IF;
            EXECUTE IMMEDIATE '
                CREATE TABLE {table_name} (
                    id VARCHAR2(32) PRIMARY KEY,
                    text CLOB,
                    metadata CLOB,
                    embedding BLOB
                )
            ';
        END;
    """, {"table_name": table_name.upper(), "owner": client.username.upper()})
    client.commit()

資料庫連接成功！


In [4]:
# 讀取PDF文檔
pdf_path = '202402_2330_AI1_20241003_160741.pdf'
try:
    pdf = PdfReader(pdf_path)
    print(f"文檔'{pdf_path}'已加載，共{len(pdf.pages)}頁。")
except Exception as e:
    print(f"無法加載PDF文檔 '{pdf_path}'：", e)
    sys.exit(1)

# 將PDF轉換為純文本
text = ""
for page_num, page in enumerate(pdf.pages, start=1):
    page_text = page.extract_text()
    if page_text:
        text += page_text
print("已將PDF文檔轉換為純文本。")

# 將文本拆分為chunks
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=800, chunk_overlap=100, length_function=len)
chunks = text_splitter.split_text(text)
print(f"已將文本分割為 {len(chunks)} 個chunks。")

# 為每個chunk創建Document
docs = [
    Document(page_content=chunk_text, metadata={'id': str(page_num), 'link': f'Page {page_num}'})
    for page_num, chunk_text in enumerate(chunks, start=1)
]
print("已為每個文本塊創建Document對象。")

# 為每個Document生成唯一的ID
unique_ids = [hashlib.sha256(metadata['id'].encode()).hexdigest()[:16].upper() for metadata in (doc.metadata for doc in docs)]

文檔'202402_2330_AI1_20241003_160741.pdf'已加載，共95頁。
已將PDF文檔轉換為純文本。
已將文本分割為 214 個chunks。
已為每個文本塊創建Document對象。


In [5]:
# 使用HuggingFaceEmbeddings模型來將文本轉換為向量
model_4db = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings_vectors = model_4db.embed_documents([doc.page_content for doc in docs])

# 將每個Document對象的數據打包成元組
docs_data = [
    (id_, doc.page_content, json.dumps(doc.metadata), array.array('f', embedding).tobytes())
    for id_, doc, embedding in zip(unique_ids, docs, embeddings_vectors)
]

# docs_data插入資料庫
with client.cursor() as cursor:
    cursor.executemany(
        f"INSERT INTO {table_name} (id, text, metadata, embedding) VALUES (:1, :2, :3, :4)",
        docs_data
    )
client.commit()
print("已將chunk向量化並儲存到Oracle資料庫中。")

已將chunk向量化並儲存到Oracle資料庫中。


In [6]:
# 用戶提問
user_question = "政府補助計畫及金額"
print("用戶問題：", user_question)

# 檢索相似文檔
query_embedding = model_4db.embed_query(user_question)

with client.cursor() as cursor:
    cursor.execute(f"SELECT text, metadata, embedding FROM {table_name}")
    rows = cursor.fetchall()

scored_docs = []
query_vector = np.array(query_embedding)
for row in rows:
    text_lob, metadata_json_lob, embedding_blob = row
    # 將CLOB轉換為字符串
    text = text_lob.read()
    # 讀取CLOB並轉換為JSON
    metadata = json.loads(metadata_json_lob.read())  
    # 處理向量資料
    embedding_bytes = embedding_blob.read()
    db_vector = np.frombuffer(embedding_bytes, dtype=np.float32).tolist()
    # 計算相似度
    db_vector_np = np.array(db_vector)
    score = np.dot(query_vector, db_vector_np)
    # 將文檔及其分數加入列表
    doc = Document(page_content=text, metadata=metadata)
    scored_docs.append((doc, score))

# 按分數排序並返回前k個文檔
scored_docs.sort(key=lambda x: x[1], reverse=True)
top_docs = [doc for doc, score in scored_docs[:4]]
# 將相似文檔組合成上下文
context = "\n".join([doc.page_content for doc in top_docs])

# 查看每個檢索到的文檔內容
for doc in top_docs:
    print("檢索結果的內容: ", doc.page_content)

用戶問題： 政府補助計畫及金額
檢索結果的內容:  政府機構不動產抵押貸款
債券                                             
 FEDERAL NATIONAL 
MORTGAGE 
ASSOCIATION  透過其他綜合損益按公
允價值衡量之金融資
產 － －    -   USD  658,944     -   USD  109,034     -   USD  75,467    USD  75,803   ( USD  336 )    -   USD  681,471  
 Federal Home Loan 
Mortgage Corporation  〃 － －    -   USD  350,328     -   USD  85,692     -   USD  53,041    USD  53,564   ( USD  523 )    -   USD  377,819  
 Government National 
Mortgage Association  〃 － －    -   USD  225,310     -   USD  75,996     -   USD  28,822    USD  29,230   ( USD  408 )    -   USD  267,948  
                                             
 政府公債 ／政府機構債券                                             
 United States 
Department of The 
Treasury  透過其他綜合損益按公
允價值衡量之金融資
檢索結果的內容:  JASM及台積電南京等子公司，因計畫於當地設廠營運而分別取得
日本及中國政府之補助款，主要用於補貼不動產、廠房及設備之購置成
本，以及建造廠房與生產營運所產生之部分成本與費用。上述計畫於
113年及 112年1月1日至 6月30日合計分別取得政府補助款 7,956,680
仟元及 10,311,820 仟元。  
二九、  現金流量資訊  
 (一) 非現金交易  
  113年1月1日 
至6月30日  112年1月1日 
至6月30日 
透過其他綜合

用戶問題： 政府補助計畫及金額

答案:
![](./image/政府補助.png)

In [7]:
# 問答提示詞模板
template = """根據以下內容回答問題：
{context}
問題：{question}"""
prompt = PromptTemplate.from_template(template)
input_prompt = prompt.format(context=context, question=user_question)

# 初始化OpenAI的GPT-4模型
llm = ChatOpenAI(model_name='gpt-4', openai_api_key=openai_api_key, temperature=0.8, max_tokens=2000)

# 生成答案
response = llm.invoke(input_prompt)
print("回答：", response.content)
print('-'*99)
print("恭喜🎉！您已成功完成了RAG應用程序，使用Oracle Database 23ai和LangChain實現了向量檢索和問答功能。")

回答： 根據提供的資訊，公司已分別從日本和中國政府取得補助款，主要用於補貼不動產、廠房及設備的購置成本，以及建造廠房與生產營運產生的部分成本與費用。該計畫於113年及112年1月1日至6月30日，合計分別取得政府補助款7,956,680仟元及10,311,820仟元。
---------------------------------------------------------------------------------------------------
恭喜🎉！您已成功完成了RAG應用程序，使用Oracle Database 23ai和LangChain實現了向量檢索和問答功能。
